# New Wrapper Class

The main wrapper class around a HDF5 file in this package is `H5File` which uses the wrapper class `H5Group` for `h5py.Group` and `H5Dataset` for wrapping `h5py.Dataset`

In [ ]:
import h5rdmtoolbox as h5tbx
import h5py

In [ ]:
class MyDataset(h5tbx.h5wrapper.h5file.H5Dataset):
    def __repr__(self):
        return f'{self.name}\n{self.attrs}'
    
    @property
    def is_2d(self):
        """returns whether dataset is two-dimensional or not"""
        return self.ndim == 2

Next we create a group class with a special method, hat returns all datasets of that group. Also the `create_dataset` method is overwritten. Take care to return `MyDataset` at the end of the method, otherwise dataset class of the parent class is taken.

In [ ]:
class MyGroup(h5tbx.h5wrapper.h5file.H5Group):
    
    def get_all_datasets(self):
        """returns all datasets of this group"""
        return [k for k in self if isinstance(self[k], h5py.Dataset)]
    
    def create_group(self, *args, **kwargs):
        return __class__(super().create_group(*args, **kwargs).id)
    
    # def create_dataset(self, *args, **kwargs):
    #     return __class__(super().create_dataset(*args, **kwargs).id)
    
    def create_dataset(self, name, user, *args, **kwargs):
        ds = super().create_dataset(name, *args, **kwargs)
        ds.attrs.modify('user', user)
        return self._h5ds(ds.id)

The main file wrapper inherites from `H5File` ("root" parent was `h5py.File`) and the new group class. Next, we have set the group and dataset class again, since some methods in the file wrapper class will need that information when returning instances of those classes (e.g. dataset or group creation). Finally we define a new method which sets the user name to the root attributes:

In [ ]:
class MyWrapper(h5tbx.H5File, MyGroup):
    
    def set_user(self, user_name):
        self.attrs.modify('user', user_name)

register the dataset and group class in all classes. This is needed, so all return objects are of the newly defined types

In [ ]:
MyGroup._h5ds = MyDataset
MyGroup._h5grp = MyGroup

MyDataset._h5ds = MyDataset
MyDataset._h5grp = MyGroup

In [ ]:
h5 = MyWrapper()

In [ ]:
type(h5)

In [ ]:
g = h5.create_group('grp', overwrite=True)

In [ ]:
type(g)

In [ ]:
gg = g.create_group('grp', overwrite=True)

In [ ]:
type(gg)

In [ ]:
ds = gg.create_dataset('hello', user='test user', shape=(2,3), units='', long_name='a long name', overwrite=True)

In [ ]:
ds

In [ ]:
type(ds)

## Accessors

In [ ]:
from h5rdmtoolbox.h5wrapper.accessory import register_special_dataset, register_special_property

In [ ]:
import xarray as xr
@register_special_dataset('user', h5tbx.h5wrapper.h5file.H5Dataset)
class InfoProperty:
    """Device Accessor class"""

    def __init__(self, ds):
        self._ds = ds
        self._username = 'NoName'

    @property
    def name(self):
        return self._username
    
    @name.setter
    def name(self, new_name):
        self._username = new_name

In [ ]:
with h5tbx.H5File() as h5:
    ds = h5.create_dataset('test', shape=(2,), units='', standard_name='standard_test')
    print(ds.user)
    print(ds.user.name)

In [ ]:
# register a property with setter and getter method:
@register_special_property(h5tbx.h5wrapper.h5file.H5Dataset, overwrite=True)
class username:
    """User Name Property"""

    def get(self):
        """geter method"""
        return self.attrs['user_name']

    def set(self, user_name: str):
        self.attrs['user_name'] = user_name

In [ ]:
with h5tbx.H5File() as h5:
    ds = h5.create_dataset('test', shape=(2,), units='', standard_name='standard_test')
    print(ds.user)
    print(ds.user.name)
    
    ds.attrs['user_name'] = 'First User'
    print(ds.username)
    ds.username = 'Second User'
    print(ds.username)
    h5.dump()